Imports

In [5]:
import f90nml
import numpy as np
from pint import UnitRegistry; AssignQuantity = UnitRegistry().Quantity
import os
import reference_solution as refsol
from scipy.fft import rfft
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import icepinn as ip

torch.set_default_dtype(torch.float64)
print(torch.cuda.device_count())
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

device = ip.get_device()

1
cuda


In [6]:
# Read in GI parameters
inputfile = "GI parameters - Reference limit cycle (for testing).nml"
GI=f90nml.read(inputfile)['GI']
nx_crystal = GI['nx_crystal']
L = GI['L']
NBAR = GI['Nbar']
NSTAR = GI['Nstar']

# Define t range (needs to be same as training file)
RUNTIME = 5
NUM_T_STEPS = RUNTIME + 1
#NUM_T_STEPS = RUNTIME*5 + 1

# Define initial conditions
Ntot_init = np.ones(nx_crystal)
Nqll_init = ip.get_Nqll(Ntot_init)

# Define x, t pairs for training
X_QLC = np.linspace(-L,L,nx_crystal)
t_points = np.linspace(0, RUNTIME, NUM_T_STEPS)
x, t = np.meshgrid(X_QLC, t_points)
training_set = torch.tensor(np.column_stack((x.flatten(), t.flatten()))).to(device)

## Naming Legend

CL = curriculum learning  
SF = SF_Pinn architecture  
HE = hard-enforced initial condition  
{n}wide = nodes per FC-layer  
nodiff = diffusion term is excluded  


In [16]:
MODEL_NAME = "CL_HE_16wide_nodiff"

# Pre-load non-IC-enforced model?
preload = True

# Define model attributes
model_dimensions = torch.tensor([8, 16]).to(device)
is_sf_PINN = torch.tensor(False)
diffusion = False

In [17]:
# instantiate model
model1 = ip.IcePINN(
	num_hidden_layers=model_dimensions[0], 
	hidden_layer_size=model_dimensions[1],
	is_sf_PINN=is_sf_PINN.item()).to(device)

# Attach model attributes as buffers so they can be saved and loaded
model1.register_buffer('dimensions', model_dimensions)
model1.register_buffer('is_sf_PINN', is_sf_PINN)

# Initialize model weights with HE initialization
model1.apply(ip.init_HE)

# Define learning rate scheduling scheme
# scheduler_summed = optim.lr_scheduler.ReduceLROnPlateau(
#         optimizer, mode='min', factor=0.5, patience=10000
#     )

IcePINN(
  (sml): SinusoidalMappingLayer()
  (post_sml): Linear(in_features=48, out_features=16, bias=True)
  (sin): SinActivation()
  (fc_in): Linear(in_features=2, out_features=16, bias=True)
  (post_fc_in): Linear(in_features=16, out_features=16, bias=True)
  (fc_hidden): ModuleList(
    (0-5): 6 x Linear(in_features=16, out_features=16, bias=True)
  )
  (fc_out): Linear(in_features=16, out_features=2, bias=True)
)

In [18]:
print(MODEL_NAME)
print(training_set.shape)
print(ip.calc_cp_loss(model1, training_set, ip.get_misc_params()).shape)
print(ip.enforced_model(training_set, model1).shape)
print(model1)

CL_HE_16wide_nodiff
torch.Size([1920, 2])
torch.Size([1920, 2])
torch.Size([1920, 2])
IcePINN(
  (sml): SinusoidalMappingLayer()
  (post_sml): Linear(in_features=48, out_features=16, bias=True)
  (sin): SinActivation()
  (fc_in): Linear(in_features=2, out_features=16, bias=True)
  (post_fc_in): Linear(in_features=16, out_features=16, bias=True)
  (fc_hidden): ModuleList(
    (0-5): 6 x Linear(in_features=16, out_features=16, bias=True)
  )
  (fc_out): Linear(in_features=16, out_features=2, bias=True)
)


Train model

In [19]:
# First, train without IC enforced (if it wasn't pre-loaded)
if not preload:
    optimizer = torch.optim.AdamW(model1.parameters(), lr=1e-5)
    ip.train_IcePINN(
        model=model1, 
        optimizer=optimizer, 
        training_set=training_set, 
        epochs=50_000, 
        name=MODEL_NAME, 
        print_every=1_000,
        diffusion=diffusion,
        LR_scheduler=None,
        enforce_IC=False)
else:
    model1 = ip.load_IcePINN(MODEL_NAME, pre_IC=True)
    model1.train()


In [20]:
# Restart optimizer
optimizer = torch.optim.AdamW(model1.parameters(), lr=1e-4)

# Then, gradually enforce IC over adjustment_period and keep it enforced
ip.train_IcePINN(
    model=model1, 
    optimizer=optimizer, 
    training_set=training_set, 
    epochs=200_000, 
    name=MODEL_NAME, 
    print_every=1_000,
    diffusion=diffusion,
    LR_scheduler=None,
    enforce_IC=True,
    adjustment_period=100_000)

Commencing PINN training on 1920 points for 200000 epochs.
IC is enforced with an adjustment period of 100000.
Epoch [1k/200k] at 0m 7s: Ntot = 0.012, Nqll = 0.006, LR = 0.0001
Epoch [2k/200k] at 0m 14s: Ntot = 0.069, Nqll = 0.020, LR = 0.0001
Epoch [3k/200k] at 0m 21s: Ntot = 0.114, Nqll = 0.031, LR = 0.0001
Epoch [4k/200k] at 0m 27s: Ntot = 0.112, Nqll = 0.035, LR = 0.0001
Epoch [5k/200k] at 0m 34s: Ntot = 0.116, Nqll = 0.034, LR = 0.0001
Epoch [6k/200k] at 0m 41s: Ntot = 0.112, Nqll = 0.035, LR = 0.0001
Epoch [7k/200k] at 0m 48s: Ntot = 0.133, Nqll = 0.039, LR = 0.0001
Epoch [8k/200k] at 0m 54s: Ntot = 1.065, Nqll = 0.273, LR = 0.0001
Epoch [9k/200k] at 1m 1s: Ntot = 0.646, Nqll = 0.265, LR = 0.0001
Epoch [10k/200k] at 1m 8s: Ntot = 0.785, Nqll = 0.241, LR = 0.0001
Epoch [11k/200k] at 1m 14s: Ntot = 0.805, Nqll = 0.217, LR = 0.0001
Epoch [12k/200k] at 1m 21s: Ntot = 0.682, Nqll = 0.171, LR = 0.0001
Epoch [13k/200k] at 1m 28s: Ntot = 0.657, Nqll = 0.158, LR = 0.0001
Epoch [14k/200k] 

In [ ]:
# Instantiate model2 as best version of model1
# model2 = ip.load_IcePINN(MODEL_NAME)
# model2.train()
# optimizer2 = torch.optim.AdamW(model2.parameters(), lr=1e-6)

Train again on smaller LR starting from saved checkpoint (best saved model above)

In [ ]:
# ip.train_IcePINN(
#     model=model1, 
#     optimizer=optimizer, 
#     training_set=training_set, 
#     epochs=100_000, 
#     name=MODEL_NAME, 
#     print_every=1_000, 
#     diffusion=diffusion,
#     LR_scheduler=None,
#     enforce_IC=True,
#     adjustment_period=0)